In [183]:
import os

# Pandas
import pandas as pd

#Matplot
import matplotlib.pyplot as plt

# WordCloud (https://github.com/amueller/word_cloud)
from wordcloud import WordCloud

#NTK
import nltk
from nltk import word_tokenize

import warnings
warnings.filterwarnings('ignore')

In [184]:
#Data location
data_path = "hillary-clinton-emails/"

capital_cities = "https://raw.githubusercontent.com/icyrockcom/country-capitals/master/data/country-list.csv"

#Import data
aliases          = pd.read_csv(data_path+"Aliases.csv",         index_col=0)
emailsReceivers  = pd.read_csv(data_path+"EmailReceivers.csv",  index_col=0)
emails           = pd.read_csv(data_path+"Emails.csv",          index_col=0)
persons          = pd.read_csv(data_path+"Persons.csv",         index_col=0)

capitals         = pd.read_csv(capital_cities)

In [185]:
emails_sub_body = emails[['ExtractedBodyText','ExtractedSubject']]
emails_sub_body.count()

ExtractedBodyText    6742
ExtractedSubject     6260
dtype: int64

In [186]:
emails_sub_body.ExtractedBodyText.fillna('',inplace=True)
emails_sub_body.ExtractedSubject.fillna('',inplace=True)
emails_sub_body["SubBody"] = emails_sub_body['ExtractedBodyText'] + " " + emails_sub_body['ExtractedSubject']

In [187]:
emails = emails_sub_body.drop(['ExtractedBodyText', 'ExtractedSubject'], 1)
emails.sample(10)

,SubBody
Id,
2852,"Also, steinmeir asking to talk to u about afgh..."
4580,Shaun just called. Vote just happened. UUP una...
3465,Yes will do. Re: Guinea
2118,"Campbell, Kurt M <CampbelIKM@state.gov>\nSatur..."
3032,"For oprah, Monday is better. rm going to confi..."
528,Thx. Re: Honduras asks us to co-sponsor GA res...
7723,BINGO! Thx--I thought we were losing our memor...
2833,Ok. Re:
7249,"RELEASE IN PART\n1.4(B),B1,1.4(D),B6 Fw:=Guida..."


In [188]:
test_sample = emails_sub_body['SubBody'].loc[345]
print(test_sample)

Here's a partial list of followup from our last trip and the last week:
What can we do to help protect the Christians in Iraq as requested by Ken Joseph whom we saw in Baghdad?
JoDee Winterhof raised questions about how the PRTs and the language DOD uses about them are problematic for
NGOs like care.
Pls ask one of Holbrooke's people if they ever talked to Wolfgang Danspeckgruber at Princeton about building a railroad
in Aghanistan.
Also Dr. Arthur Keys at International Relief + Development wanted to talk w someone from Holbrooke's team about
development in Af.
I asked the Spec IG for Af Recon, Arnold Fields, to alert us to problems as soon as they can. I'm not sure how to formalize
this or even if it's appropriate. Let's discuss.
What are the "Iran Watchers"? Followup


In [189]:
import pycountry

all_country = []

for c in list(pycountry.countries):
    country_entry = [c.alpha_2, c.alpha_3, c.name, c.numeric, getattr(c, 'official_name', "")]
    all_country.append(country_entry)
    
country_dict = pd.DataFrame(all_country, columns=('Alpha_2', 'Alpha3', 'Name', 'Numeric', 'Official_name'))

country_dict.head()

,Alpha_2,Alpha3,Name,Numeric,Official_name
0,AW,ABW,Aruba,533,
1,AF,AFG,Afghanistan,004,Islamic Republic of Afghanistan
2,AO,AGO,Angola,024,Republic of Angola
3,AI,AIA,Anguilla,660,
4,AX,ALA,Åland Islands,248,


In [190]:
capitals.head()

,country,capital,type
0,Abkhazia,Sukhumi,countryCapital
1,Afghanistan,Kabul,countryCapital
2,Akrotiri and Dhekelia,Episkopi Cantonment,countryCapital
3,Albania,Tirana,countryCapital
4,Algeria,Algiers,countryCapital


In [191]:
country_dict['Capital'] = ""

for i, capital_entry in capitals.iterrows():
    for j, country_entry in country_dict.iterrows():
        if (capital_entry['country'] == country_entry['Name']):
            
            country_dict.loc[j]['Capital'] = capital_entry['capital']

country_dict.head()

,Alpha_2,Alpha3,Name,Numeric,Official_name,Capital
0,AW,ABW,Aruba,533,,Oranjestad
1,AF,AFG,Afghanistan,004,Islamic Republic of Afghanistan,Kabul
2,AO,AGO,Angola,024,Republic of Angola,Luanda
3,AI,AIA,Anguilla,660,,The Valley
4,AX,ALA,Åland Islands,248,,


In [192]:
country_dict['Alt_names'] = ""

country_dict.head()

,Alpha_2,Alpha3,Name,Numeric,Official_name,Capital,Alt_names
0,AW,ABW,Aruba,533,,Oranjestad,
1,AF,AFG,Afghanistan,004,Islamic Republic of Afghanistan,Kabul,
2,AO,AGO,Angola,024,Republic of Angola,Luanda,
3,AI,AIA,Anguilla,660,,The Valley,
4,AX,ALA,Åland Islands,248,,,


In [199]:
# check if the text content mentions any country according to our dictionary
def containsCountryInfo(content):
    country_list = []
    
    for index, row in country_dict.iterrows():
        name_list = country_city_list(index)
        
        for name in name_list:
            if(name != "" and name in content):
                country_list.append(name_list[0])
                
    return country_list

In [194]:
# extract useful info from our dictionay into list
def country_city_list(n):
    l = []
    country_entry = country_dict.loc[n]
    
    l.append(country_entry.Name)
    
    if (country_entry.Capital != ""):
        l.append(country_entry.Capital)
    
    if (country_entry.Alt_names != ""):
        names = country_entry.Alt_names.split("-")
        l.extend(names)
        
    return l

country_city_list(0)

['Aruba', 'Oranjestad']

In [195]:
# function to add any alternative name to a country
def add_country_alt_name(name, alt):
    for index, row in country_dict.iterrows():
        if(row.Name == name):
            row.Alt_names += "-"
            row.Alt_names += alt
            print("Added successfully")

In [196]:
# TEST

add_country_alt_name("Aruba", "Danny")

s = "Danny sdafsa dsagsdg"

Added successfully


In [201]:
print(containsCountryInfo(s))

containsCountryInfo(test_sample)

['Aruba']


['Iraq', 'Iraq']